# Pittsburgh's Next Top Neighborhood

## 🚗 Commutability

In [2]:
import pandas as pd

# read census data
data = pd.read_csv("data/SNAP/Neighborhoods_with_SNAP_data.csv")

# make easier column names
data['name'] = data['Neighborhood_2010_HOOD']             # neighborhood name
data['drive'] = data['SNAP_All_csv_Commute_to_Work__D']   # drive to work alone
data['carpool'] = data['SNAP_All_csv_Commute_to_Work__C'] # carpool/vanpool
data['public'] = data['SNAP_All_csv_Commute_to_Work__P']  # public transit
data['taxi'] = data['SNAP_All_csv_Commute_to_Work__T']    # taxi
data['motor'] = data['SNAP_All_csv_Commute_to_Work__M']   # motorcycle
data['bike'] = data['SNAP_All_csv_Commute_to_Work__B']    # bike
data['walk'] = data['SNAP_All_csv_Commute_to_Work__W']    # walk
data['other'] = data['SNAP_All_csv_Commute_to_Work__O']   # other

# clean data
data = data.fillna(0)

# constants to determine value of method
# best values are 1, okay are 0.5, bad or null are 0.25
good = 1   # bike, walk
okay = 0.5 # carpool, public transit
bad = 0.25 # drive, taxi, motorcycle, other

# convert percent value to float
def convert(value):
    value = str(value)
    value = value.replace('%','')
    return float(value)

# iterate through rows, calculate commutability score
for idx, row in data.iterrows():
    score = convert(data.loc[idx,'bike']) + convert(data.loc[idx, 'walk']) # good methods
    score += okay*(convert(data.loc[idx, 'carpool']) + convert(data.loc[idx, 'public'])) # okay methods
    score += bad*(convert(data.loc[idx, 'drive']) + convert(data.loc[idx, 'taxi']) + convert(data.loc[idx, 'motor']) + convert(data.loc[idx, 'other'])) # bad methods
    data.at[idx,'raw'] = score # set neighborhood's commutability score

# find score using Pawel's formula
data['score'] = data['raw']/data['raw'].max()

# view results
commute = data[['name','score']]
commute

,name,score
0,Central Oakland,1.000000
1,North Oakland,0.916748
2,West Oakland,0.801828
3,Crawford-Roberts,0.475351
4,Strip District,0.418544
...,...,...
85,Chartiers City,0.399935
86,Ridgemont,0.487757
87,West End,0.433562
88,California-Kirkbride,0.594842
